# Add libraries

In [1]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import models

In [3]:
abs_path = '/content/drive/MyDrive/Datasets/CMPE468/sign-dataset/'

# Analyze Dataset

In [4]:
all_in_one = pd.read_csv(os.path.join(abs_path, 'annotations.csv'))

# Uncomment below code if you want to check dataset images.
# seen_classes = []
# for index, row in all_in_one.iterrows():
#   i_class = row.category
#   if i_class not in seen_classes:
#     print(i_class)
#     seen_classes.append(i_class)
#     image = Image.open(os.path.join(abs_path, 'images', row.file_name))
#     plt.imshow(image)
#     plt.show()

In [5]:
# Selected class 3 -> 40 Speed Sign - 318
# Selected class 16 -> No Car Sign - 218
# Selected class 17 -> No Horn Sign - 192
# Selected class 30 -> Bike Sign - 184
# Selected class 54 -> No Park Sign - 500
# Selected class 55 -> 40 Speed Sign - 220

# Dataset Distribution 140 Train 20 Test 20 Validation due to minimum class that has selected.

selected_classes = [3, 16, 17, 30, 54, 55]
all_in_one_selected = all_in_one[all_in_one.category.isin(selected_classes)]
all_in_one_selected.drop_duplicates(subset=['file_name'], inplace=True)

<ipython-input-5-a5f3f2ffbd16>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_in_one_selected.drop_duplicates(subset=['file_name'], inplace=True)


# Training Test and Validation Dataframes

## Create OneHotLabel

In [6]:
def change_to_onehot(label):
  if label == selected_classes[0]:
    return '[1, 0, 0, 0, 0, 0]'
  elif label == selected_classes[1]:
    return '[0, 1, 0, 0, 0, 0]'
  elif label == selected_classes[2]:
    return '[0, 0, 1, 0, 0, 0]'
  elif label == selected_classes[3]:
    return '[0, 0, 0, 1, 0, 0]'
  elif label == selected_classes[4]:
    return '[0, 0, 0, 0, 1, 0]'
  elif label == selected_classes[5]:
    return '[0, 0, 0, 0, 0, 1]'

## Train - Test - Val Dataset Creation


In [7]:
train_list = []
test_list = []
val_list = []

train_limit = 140
test_limit = 20
val_limit = 20

for selected_class in selected_classes:
  inner_df = all_in_one_selected[all_in_one_selected.category == selected_class]

  selected_train = 0
  selected_test = 0
  selected_val = 0

  for i_index, i_row in inner_df.iterrows():
    if selected_train < train_limit:
      selected_train += 1
      train_list.append(i_row)
    elif selected_test < test_limit:
      selected_test += 1
      test_list.append(i_row)
    elif selected_val < val_limit:
      selected_val += 1
      val_list.append(i_row)

train_df = pd.DataFrame(train_list)
test_df = pd.DataFrame(test_list)
val_df = pd.DataFrame(val_list)

In [8]:
 # To check all classes distributed equally.
for selected_class in selected_classes:
  assert len(train_df[train_df.category == selected_class]) == train_limit
  assert len(test_df[test_df.category == selected_class]) == test_limit
  assert len(val_df[val_df.category == selected_class]) == val_limit

### Train

In [9]:
train = train_df.reset_index()

In [10]:
train['OneHotLabel'] = train.apply(lambda x: change_to_onehot(x.category), axis=1)

In [11]:
train.groupby('OneHotLabel').agg('count')

,index,file_name,width,height,x1,y1,x2,y2,category
OneHotLabel,,,,,,,,,
"[0, 0, 0, 0, 0, 1]",140,140,140,140,140,140,140,140,140
"[0, 0, 0, 0, 1, 0]",140,140,140,140,140,140,140,140,140
"[0, 0, 0, 1, 0, 0]",140,140,140,140,140,140,140,140,140
"[0, 0, 1, 0, 0, 0]",140,140,140,140,140,140,140,140,140
"[0, 1, 0, 0, 0, 0]",140,140,140,140,140,140,140,140,140
"[1, 0, 0, 0, 0, 0]",140,140,140,140,140,140,140,140,140


### Test

In [12]:
test = test_df.reset_index()

In [13]:
test['OneHotLabel'] = test.apply(lambda x: change_to_onehot(x.category), axis=1)

In [14]:
test.groupby('OneHotLabel').agg('count')

,index,file_name,width,height,x1,y1,x2,y2,category
OneHotLabel,,,,,,,,,
"[0, 0, 0, 0, 0, 1]",20,20,20,20,20,20,20,20,20
"[0, 0, 0, 0, 1, 0]",20,20,20,20,20,20,20,20,20
"[0, 0, 0, 1, 0, 0]",20,20,20,20,20,20,20,20,20
"[0, 0, 1, 0, 0, 0]",20,20,20,20,20,20,20,20,20
"[0, 1, 0, 0, 0, 0]",20,20,20,20,20,20,20,20,20
"[1, 0, 0, 0, 0, 0]",20,20,20,20,20,20,20,20,20


### Validation

In [15]:
val = val_df.reset_index()

In [16]:
val['OneHotLabel'] = val.apply(lambda x: change_to_onehot(x.category), axis=1)

In [17]:
val.groupby('OneHotLabel').agg('count')

,index,file_name,width,height,x1,y1,x2,y2,category
OneHotLabel,,,,,,,,,
"[0, 0, 0, 0, 0, 1]",20,20,20,20,20,20,20,20,20
"[0, 0, 0, 0, 1, 0]",20,20,20,20,20,20,20,20,20
"[0, 0, 0, 1, 0, 0]",20,20,20,20,20,20,20,20,20
"[0, 0, 1, 0, 0, 0]",20,20,20,20,20,20,20,20,20
"[0, 1, 0, 0, 0, 0]",20,20,20,20,20,20,20,20,20
"[1, 0, 0, 0, 0, 0]",20,20,20,20,20,20,20,20,20


### Additional Tests

In [18]:
# To check all file_name's unique in all datasets

for _, row_train in train.iterrows():
    file_name = row_train.file_name

    matches_test = test[test.file_name == file_name]
    matches_val = val[val.file_name == file_name]

    if len(matches_test) > 0 or len(matches_val) > 0:
        print(matches_val, file_name)
        assert False

for _, row_test in test.iterrows():
    file_name = row_test.file_name

    matches_train = train[train.file_name == file_name]
    matches_val = val[val.file_name == file_name]

    if len(matches_train) > 0 or len(matches_val) > 0:
        assert False

for _, row_val in val.iterrows():
    file_name = row_val.file_name

    matches_train = train[train.file_name == file_name]
    matches_test = test[test.file_name == file_name]

    if len(matches_train) > 0 or len(matches_test) > 0:
        assert False

# Create dataset class

In [19]:
class ImageDataset(Dataset):

    def __init__(self, metadata, transform=None):
        """Init function should not do any heavy lifting, but
            must initialize how many items are available in this data set.
        """
        self.metadata = metadata
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        """ Here we have to return the item requested by `idx`
            The PyTorch DataLoader class will use this method to make an iterable for
            our training or validation loop.
        """
        image_path = self.metadata.loc[idx, 'file_name']
        image_label = self.metadata.loc[idx, 'OneHotLabel']
        image = Image.open(os.path.join(abs_path, 'images', image_path))

        image = self.transform(image)

        return image, image_label

In [20]:
input_size = 224

data_transforms = {
    'train': [
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ],
    'tv': [
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ],
}

# Apply Transforms - Prepare Datasets

In [21]:
# Define transformations (if any)
transform_train = transforms.Compose(data_transforms['train'])
transform_tv = transforms.Compose(data_transforms['tv'])

# Batch size
bs = 128

# Instantiate the dataset
train_dataset = ImageDataset(metadata=train, transform=transform_train)

# Use DataLoader to access individual samples
train_data_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)

# Instantiate the dataset
test_dataset = ImageDataset(metadata=test, transform=transform_tv)

# Use DataLoader to access individual samples
test_data_loader = DataLoader(test_dataset, batch_size=bs, shuffle=True)

# Instantiate the dataset
val_dataset = ImageDataset(metadata=val, transform=transform_tv)

# Use DataLoader to access individual samples
val_data_loader = DataLoader(val_dataset, batch_size=bs, shuffle=True)

# Training

In [22]:
# define hyperparameters
learning_rate = 0.0001
# batch_size = 32
old_loss = 100

# To select a GPU
device = torch.device('cuda')

# To clear CUDA memory
torch.cuda.empty_cache()

# Initialize the EfficientNet model
efficientnet_b0 = models.efficientnet_b0(weights='IMAGENET1K_V1')
efficientnet_b0.classifier[1] = torch.nn.Linear(in_features=1280, out_features=6, bias=True)
model = efficientnet_b0.to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Epoch size
num_epochs = 100

# Softmax
softmax = torch.nn.Softmax(dim=1)

In [23]:
for epoch in range(num_epochs):
    print(epoch+1)
    train_loss = []
    dice_train = []
    model.train()

    for batch_index_train, batch_train in enumerate(train_data_loader):
      image, image_label = batch_train
      image_label = [eval(inner_list_str) for inner_list_str in list(image_label)]
      image_label = torch.tensor(image_label, dtype=torch.float32).to(device)

      image = image.to(device)
      outputs = model(image)
      outputs = softmax(outputs)

      loss = criterion(outputs, image_label)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    with torch.no_grad():
      model.eval()
      for batch_index_val, batch_val in enumerate(val_data_loader):
        image, image_label = batch_val
        image_label = [eval(inner_list_str) for inner_list_str in list(image_label)]
        image_label = torch.tensor(image_label, dtype=torch.float32).to(device)

        image = image.to(device)
        outputs = model(image)
        outputs = softmax(outputs)
        loss = criterion(outputs, image_label)

      if loss < old_loss:
        old_loss = loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/Datasets/CMPE468/other.pth')
        print(loss.item())

1
1.7278989553451538
2
1.5658550262451172
3
1.4157911539077759
4
1.3193295001983643
5
1.22444748878479
6
1.1458017826080322
7
1.1118018627166748
8
1.0979225635528564
9
1.093312382698059
10
1.0904291868209839
11
1.0874009132385254
12
1.0825661420822144
13
1.0780192613601685
14
15
1.076162338256836
16
17
1.0746190547943115
18
19
1.0737011432647705
20
1.071779727935791
21
1.070613145828247
22
23
24
25
1.0674184560775757
26
1.0668776035308838
27
1.065284252166748
28
1.0639771223068237
29
1.062424659729004
30
1.0601022243499756
31
1.0597705841064453
32
1.0588033199310303
33
1.058237075805664
34
1.0576609373092651
35
36
1.0575902462005615
37
1.0568710565567017
38
39
1.0564483404159546
40
41
1.056006669998169
42
43
44
1.055299997329712
45
46
47
1.054599404335022
48
49
50
51
52
1.0544912815093994
53
1.0541956424713135
54
1.0533368587493896
55
1.053256869316101
56
1.0528895854949951
57
1.0526195764541626
58
59
60
61
62
1.05259108543396
63
64
65
66
67
68
69
1.0514904260635376
70
1.05138814449310

# Testing

In [40]:
# To clear CUDA memory
torch.cuda.empty_cache()

test_efficient = models.efficientnet_b0(weights=None)
test_efficient.classifier[1] = torch.nn.Linear(in_features=1280, out_features=6, bias=True)
test_model = test_efficient.to(device)
test_model.load_state_dict(torch.load('/content/drive/MyDrive/Datasets/CMPE468/other.pth'))

<All keys matched successfully>

In [43]:
test_model.eval()

total = len(test)
correctly_predicted = 0

with torch.no_grad():
  for batch_index_test, batch_test in enumerate(test_data_loader):
      image, image_label = batch_test
      image_label = [eval(inner_list_str) for inner_list_str in list(image_label)]
      image_label = torch.tensor(image_label, dtype=torch.float32).to(device)

      image = image.to(device)
      outputs = test_model(image)
      outputs = softmax(outputs)
      loss = criterion(outputs, image_label)

      index_predicted = torch.argmax(outputs, dim=1, keepdim=True).to('cpu').numpy()

      for index, label in enumerate(image_label):
        label = label.to('cpu').numpy()
        if label[index_predicted[index]] == 1:
          correctly_predicted += 1

      # print(index_predicted, image_label)

print((correctly_predicted / total) * 100, loss)

100.0 tensor(1.0471, device='cuda:0')
